In [1]:
!git clone https://github.com/hygo2025/cdd-projeto-final.git

Cloning into 'cdd-projeto-final'...
remote: Enumerating objects: 173, done.
remote: Counting objects: 100% (173/173), done.
remote: Compressing objects: 100% (105/105), done.
remote: Total 173 (delta 87), reused 142 (delta 60), pack-reused 0 (from 0)
Receiving objects: 100% (173/173), 62.73 KiB | 1.39 MiB/s, done.
Resolving deltas: 100% (87/87), done.


In [2]:
# Install java
!apt install openjdk-21-jdk

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  fonts-dejavu-core fonts-dejavu-extra libatk-wrapper-java libatk-wrapper-java-jni libfontenc1
  libice-dev libsm-dev libxkbfile1 libxt-dev libxtst6 libxxf86dga1 openjdk-21-jdk-headless
  openjdk-21-jre openjdk-21-jre-headless x11-utils
Suggested packages:
  libice-doc libsm-doc libxt-doc openjdk-21-demo openjdk-21-source visualvm libnss-mdns
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei | fonts-wqy-zenhei fonts-indic
  mesa-utils
The following NEW packages will be installed:
  fonts-dejavu-core fonts-dejavu-extra libatk-wrapper-java libatk-wrapper-java-jni libfontenc1
  libice-dev libsm-dev libxkbfile1 libxt-dev libxtst6 libxxf86dga1 openjdk-21-jdk
  openjdk-21-jdk-headless openjdk-21-jre openjdk-21-jre-headless x11-utils
0 upgraded, 16 newly installed, 0 to remove and 29 not upgraded.
Need to get 135 MB of archives.
Aft

In [3]:
!wget "https://repo1.maven.org/maven2/org/apache/hadoop/hadoop-aws/3.3.4/hadoop-aws-3.3.4.jar" -P "spark/lib-jars/"
!wget "https://repo1.maven.org/maven2/com/amazonaws/aws-java-sdk-bundle/1.12.699/aws-java-sdk-bundle-1.12.699.jar" -P "spark/lib-jars/"
!wget "https://repo1.maven.org/maven2/com/amazonaws/aws-java-sdk/1.12.699/aws-java-sdk-1.12.699.jar" -P "spark/lib-jars/"
!wget "https://repo1.maven.org/maven2/com/amazonaws/aws-java-sdk-core/1.12.699/aws-java-sdk-core-1.12.699.jar" -P "spark/lib-jars/"
!wget "https://repo1.maven.org/maven2/com/amazonaws/aws-java-sdk-s3/1.12.699/aws-java-sdk-s3-1.12.699.jar" -P "spark/lib-jars/"

--2025-03-16 18:52:41--  https://repo1.maven.org/maven2/org/apache/hadoop/hadoop-aws/3.3.4/hadoop-aws-3.3.4.jar
Resolving repo1.maven.org (repo1.maven.org)... 199.232.192.209, 199.232.196.209, 2a04:4e42:4c::209, ...
Connecting to repo1.maven.org (repo1.maven.org)|199.232.192.209|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 962685 (940K) [application/java-archive]
Saving to: ‘spark/lib-jars/hadoop-aws-3.3.4.jar’

hadoop-aws-3.3.4.ja 100%[===================>] 940.12K  1.39MB/s    in 0.7s    

2025-03-16 18:52:42 (1.39 MB/s) - ‘spark/lib-jars/hadoop-aws-3.3.4.jar’ saved [962685/962685]

--2025-03-16 18:52:42--  https://repo1.maven.org/maven2/com/amazonaws/aws-java-sdk-bundle/1.12.699/aws-java-sdk-bundle-1.12.699.jar
Resolving repo1.maven.org (repo1.maven.org)... 199.232.192.209, 199.232.196.209, 2a04:4e42:4c::209, ...
Connecting to repo1.maven.org (repo1.maven.org)|199.232.192.209|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 37648

In [4]:
# configurar as variáveis de ambiente
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-21-openjdk-amd64"

os.environ["SPARK_DRIVER_EXTRACLASSPATH"] = "/content/spark/lib-jars/*"
os.environ["SPARK_EXECUTOR_EXTRACLASSPATH"] = "/content/spark/lib-jars/*"
os.environ["SPARK_SERIALIZER"] = "org.apache.spark.serializer.KryoSerializer"
os.environ["ENV"] = "dev"
os.environ["PYTHONUNBUFFERED"] = "1"
os.environ["SPARK_EXECUTOR_MEMORY"] = "8g"
os.environ["SPARK_DRIVER_MEMORY"] = "8g"
os.environ["SPARK_MEMORY_FRACTION"] = "8g"


os.environ["DOWNLOAD_FOLDER"] = "/content/datasets"

In [5]:
!java --version

openjdk 21.0.6 2025-01-21
OpenJDK Runtime Environment (build 21.0.6+7-Ubuntu-122.04.1)
OpenJDK 64-Bit Server VM (build 21.0.6+7-Ubuntu-122.04.1, mixed mode, sharing)


In [6]:
%load_ext autoreload
%autoreload 2

In [7]:
try:  # When on Google Colab, clone the repository to download any necessary cache.
    import google.colab
    repo_path = 'cdd'
    !git -C $repo_path pull origin || git clone https://github.com/hygo2025/cdd-projeto-final.git $repo_path
except:
    repo_path = '.'  # Use the local path if not on Google Colab

fatal: cannot change to 'cdd': No such file or directory
Cloning into 'cdd'...
remote: Enumerating objects: 173, done.
remote: Counting objects: 100% (173/173), done.
remote: Compressing objects: 100% (105/105), done.
remote: Total 173 (delta 87), reused 142 (delta 60), pack-reused 0 (from 0)
Receiving objects: 100% (173/173), 62.73 KiB | 868.00 KiB/s, done.
Resolving deltas: 100% (87/87), done.


In [8]:
import sys
import os

if repo_path not in sys.path:
    sys.path.append(repo_path)

In [ ]:
!pip install -r cdd/requirements.txt

Ignoring spacy: markers 'python_version <= "3.8"' don't match your environment
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 6.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.7/40.7 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 4.4 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of tensorflow[and-cuda] to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of tensorflow[and-cuda] to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking fo

In [33]:
import pkg_resources
!pip install -e $repo_path

Obtaining file:///content/cdd
  Preparing metadata (setup.py) ... done
  Attempting uninstall: cdd
    Found existing installation: cdd 0.1.0
    Uninstalling cdd-0.1.0:
      Successfully uninstalled cdd-0.1.0
  Running setup.py develop for cdd


In [ ]:
import cdd

In [ ]:
from src.sar_model import SarModel
from src.utils.enums import MovieLensDataset, SimilarityType, MovieLensType

In [ ]:
from src.sar_model import SarModel
from src.utils.enums import MovieLensDataset, SimilarityType

sar_model = SarModel(
  dataset=MovieLensDataset.ML_1M,
  top_k=10,
  validate_size=0.25,
  time_decay_coefficient=30,
  similarity_type=SimilarityType.COSINE,
  seed=42
)
sar_model.train()
result = sar_model.evaluate()
print("Resultados da avaliação:")
print(result)